In [23]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

    
    
    
def Belief(State,p,r):
   
    equilibrium = r/(1-p+r);
    
    if State[0]== 0 and State[1]> 0:
            Belief = (r-r*(p-r)**(State[1]))/(1+r-p);
    elif State[0]== 1 and State[1]> 0:
            Belief = (r+(1-p)*(p-r)**(State[1]))/(1+r-p);
            
    else:
        
        Belief = equilibrium;

    return Belief;
            
        
def WhittleIndex(State,p,r):
    WhittleIndex = 0.;

    
    if State[0]== 0 and State[1]>0:
        l= State[1];
        x= Belief([0,l],p,r);
        y=Belief([0,l+1],p,r);
        WhittleIndex =((x-y)*(l+1)+y)/(1-p+(x-y)*l+y); 
    elif State[0] == 1:
        WhittleIndex = r/((1-p)*(1+r-p)+r);
    
    else:
        WhittleIndex = r/((1-p)*(1+r-p)+r);
            
    return WhittleIndex;

def update(S,C,Reward,alpha,p,r,q,s,gamma,N):
        #the update process of each timestep.
        
        #S is the list of belief states of all channels.
        #C is the list of actual channel states.
        W = [0.]*N;
        Y = [0.]*N;
        Action = [0]*N;
        Threshold = 0.;
        Seed = np.random.rand(N);
        
       #Get the Threshold. 
        for i in range(int(N*gamma)):
            Y[i]= WhittleIndex(S[i],p,r);
        for i in range(int(N*gamma),N):
            Y[i]= WhittleIndex(S[i],q,s);
        
        Y.sort();
        Threshold = Y[int((1-alpha)*N)];
        
        
        
       #Get the Action vector.
        for i in range(int(N*gamma)):
            W[i]= WhittleIndex(S[i],p,r);
        for i in range(int(N*gamma),N):
            W[i]= WhittleIndex(S[i],q,s);
    
        h=0;
        for i in range(N):
            if W[i]>= Threshold:
                Action[i]=1;
                h=h+1;
            if h >= alpha*N:
                break;
            
       #The update process.
       
        for i in range(int(N*gamma)):
        
            if Action[i] == 0 and S[i][1]>0:
                S[i][1]= S[i][1]+1;
                if C[i]==0:
                    if Seed[i]<r:
                        C[i]=1;
                    else:
                        C[i]=0;
                else:
                    if Seed[i]<p:
                        C[i]=1;
                    else:
                        C[i]=0;
                
                
                
               
             
            elif Action[i] == 0 and S[i][1]==0:
                if C[i]==0:
                    if Seed[i]<r:
                        C[i]=1;
                    else:
                        C[i]=0;
                else: 
                    if Seed[i]<p:
                        C[i]=1;
                    else:
                        C[i]=0;
            
            else:
                if C[i]==1:
                    S[i]= [1,1];
                    Reward = Reward + 1;
                    C[i]= int((Seed[i]<p)==True);
                    
                else:
                    S[i]= [0,1];
                    C[i]= int((Seed[i]<r)==True);
        
        for i in range(int(N*gamma),N):
        
            if Action[i] == 0 and S[i][1]>0:
                S[i][1]= S[i][1]+1;
                if C[i]==0:
                    if Seed[i]<s:
                        C[i]=1;
                    else:
                        C[i]=0;
                else:
                    if Seed[i]<q:
                        C[i]=1;
                    else:
                        C[i]=0;
                
                
                
               
             
            elif Action[i] == 0 and S[i][1]==0:
                if C[i]==0:
                    if Seed[i]<s:
                        C[i]=1;
                    else:
                        C[i]=0;
                else: 
                    if Seed[i]<q:
                        C[i]=1;
                    else:
                        C[i]=0;
            
            else:
                if C[i]==1:
                    S[i]= [1,1];
                    Reward = Reward + 1;
                    C[i]= int((Seed[i]<q)==True);
                    
                else:
                    S[i]= [0,1];
                    C[i]= int((Seed[i]<s)==True);
        
        
        
        return Reward;
    


In [9]:
print(update([[0,0],[0,0],[0,3],[0,2],[0,0],[1,4],[1,3],[1,7],[0,4],[0,6]],[0,0,1,0,0,1,1,0,0,0],0,0.5,0.4,0.3,0.6,0.2,0.6,10));

2


In [24]:
def onetrial(S,C,Reward,timesteps,alpha,p,r,q,s,gamma,N):

    for i in range(timesteps):
        Reward = update(S,C,Reward,alpha,p,r,q,s,gamma,N);
    return Reward;

In [25]:
print(onetrial([[0,0],[0,0],[0,3],[0,2],[0,0],[1,4],[1,3],[1,7],[0,4],[0,6]],[0,0,1,0,0,1,1,0,0,0],0,10,0.5,0.4,0.3,0.6,0.2,0.6,10));

12


In [26]:
def mc(S,Reward,ntrials,timesteps,alpha,p,r,q,s,gamma,N):
    e_1 = r/(1-p+r);
    e_2 = s/(1-q+s);
    
   
    bigS = [S]*(ntrials);
    for i in range(ntrials):
        C = [0]*N;
        Seed = np.random.rand(N);
        for j in range(int(N*gamma)):
            C[j]= int((Seed[j]< e_1) == True);
            
        for j in range(int(N*gamma),N):
            C[j]= int((Seed[j]<e_2) == True);
    
        Reward = onetrial(bigS[i],C,Reward,timesteps,alpha,p,r,q,s,gamma,N);
        
    lowerbound = 100*Reward/(timesteps*ntrials*N);
    
    print(lowerbound);


In [27]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.5,0.4,0.3,0.6,0.2,0.6,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];

#For this set of parameters, the performance of the two policies are nearly the same, but we clearly see
#the increasing tendance of the performance with N, the toral number of channels!

#The upperbound calculated is 19.338
    

18.57298
18.48417
18.48102
18.49588
18.512772
18.602192
18.677435


In [28]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.7,0.9,1/15,0.5,1/3,0.5,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];

#For this set of parameters, the performance of the two policies are still nearly the same,
#beacause alpha = 0.7 is large and a large portion of channel will be in Belief State [1,1], so
#the two policies will not distinguish between them!

#But still, we observe the increasing of performance with N, the total number of channels!
#That's a good news!!!

#The upperbound calculated is 35.288
    

34.21854
34.38348
34.44988
34.486515
34.554204
34.720798
34.821584


In [29]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.2,0.65,0.15,0.44,0.24,0.5,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];

#For this set of parameters, WhittleIndex Policy performs much better than the BeliefGreedy policy!
#Because the alpha = 0.2 is small and most of the channel will in Belief States [0,*], and we see 
#from the WhittleIndex Plot that there is a 'cross' just after 3 units of time!


#The upperbound calculated is 8.802

8.44704
8.40003
8.39524
8.395085
8.39588
8.404906
8.43877


In [30]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.2,0.9,1/15,0.5,1/3,0.5,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];

#To again verify my theory, I retest the second example with an alpha of 0.2 instead of 0.7 while keeping the 
#other parameters unchanged, to see if we will make a bigger performance difference in the two policies.

#The upperbound calculated is 14.756


13.97262
14.10359
14.14778
14.13342
14.154176
14.15611
14.172237


In [ ]:
#It's nearly 4 percent of performence improvement compare to the BeliefGreedy!!!!!!